In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout
import pandas as pd
import dataset
import seaborn as sns
import qgrid
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
qgrid.enable(dataframe=True, series=True)

In [ ]:
test = widgets.Text()
output = widgets.Text(value=test.value)
widgets.jslink((test, 'value'),(output,'value'))
widgets.HBox([test,output])

In [ ]:
a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.jslink((a, 'value'), (b, 'value'))

In [ ]:
db = dataset.connect('sqlite:///data/serienjunkies.db')

query = """
select "date",
    page,
    query,
    sum(clicks) as clicks,
    sum(impressions) as impressions,
    sum(impressions * position) as pos_imp
from web_country_device_page_query
where "date" > '2019-01-01'
group by "date", page, query
"""

df = pd.read_sql(query, con=db.engine, parse_dates=['date'])

In [ ]:
# %load -r 4:28 helpers/helpers.py
def assign_pos_imp(df):
  """Calculate pos_imp for weighted position
  
  Arguments:
    df -- source DataFrame
  """
  return df.assign(pos_imp=lambda x: x.position * x.impressions)

def assign_position(df):
  """Calculate position and assign new col "position"
  
  Arguments:
    df -- source DataFrame
  """
  return df.assign(position=lambda x: x.pos_imp / x.impressions)


def assign_ctr(df):
  """calculate ctr and assign new col "position"
  
  Arguments:
    df -- source DataFrame
  """
  return df.assign(ctr= lambda x: x.clicks / x.impressions)

In [ ]:
def filter_df(p_pattern, q_pattern):
    return df[(df['page'].str.contains(p_pattern, case=False, regex=True)) & (df['query'].str.contains(q_pattern, case=False, regex=True))] \
        .groupby(['page','query'], as_index=False).sum() \
        .pipe(assign_position) \
        .drop('pos_imp', axis=1) \
        .pipe(assign_ctr) \
        .sort_values('clicks', ascending=False) \
        .iloc[0:20,:]

In [ ]:
out = widgets.Output()

In [ ]:
def button_clicked(b):
    out.clear_output()
    with out:
        df[(df['page'].str.contains(page.value, case=False, regex=True)) & (df['query'].str.contains(query.value, case=False, regex=True))] \
                .groupby('date', as_index=False).sum() \
                .pipe(assign_position) \
                .drop('pos_imp', axis=1) \
                .pipe(assign_ctr) \
                .melt(id_vars = 'date', value_vars=['clicks','impressions','ctr','position']) \
                .pipe((sns.FacetGrid, 'data'), col='variable', col_wrap=2, aspect=3, sharey=False, hue='variable') \
                .map(sns.lineplot, 'date', 'value')
        plt.show()
        display(filter_df(p_pattern = page.value, q_pattern = query.value))

In [ ]:
row_layout = Layout(display='flex',flex_flow='row',align_items='stretch',width='100%')

query = widgets.Text(description='query')
page = widgets.Text(description='page', layout=Layout(flex='2 1 auto', width='auto'))
button = widgets.Button(description='show', layout=Layout(flex='2 1 auto', width='auto'))

button.on_click(button_clicked)

widgets.VBox([widgets.HBox([query,page], layout=row_layout),
              widgets.HBox([button], layout=row_layout),
              out])